# Настройка среды

In [1]:
import os
import sys
sys.path.append(os.path.abspath("../../../.."))

In [2]:
import optuna
import torch
import numpy
import json
import pickle
from typing import Literal, Union, Any
from belashovplot import TiledPlot

from elements.propagators import FurrierPropagation
from elements.modulators import PhaseModulator, AmplitudeModulator
from elements.composition import CompositeModel
from elements.detectors import ClassificationDetectors
from elements.simple import IntensityToAmplitude, AmplitudeToIntensity

from utilities.filters import Gaussian, Window
from utilities.datasets import Dataset
from utilities.losses import Normalizable, Normalization, LossLinearCombination
import cluster

In [3]:
from cluster import epochs, SelectedGPUs
ExcludedGPUs = []
GPUCount = len(SelectedGPUs.ids) - len(ExcludedGPUs)
GPUCount

8

In [4]:
import pandas
pandas.options.display.max_columns = None

# Класс настройщик вариации

In [5]:
class Variation:
    class Abstract:
        def __init__(self, name:str):
            self._name = name
        def __call__(self, trial:optuna.trial):
            raise NotImplementedError
    class Float(Abstract):
        def __init__(self, name:str, v0:float, v1:float=None):
            super().__init__(name)
            if v1 is None: v1 = v0
            self._limits = (v0, v1)
        def __call__(self, trial:optuna.trial):
            return trial.suggest_float(self._name, *self._limits)
    class Int(Abstract):
        def __init__(self, name:str, v0:int, v1:int=None):
            super().__init__(name)
            if v1 is None: v1 = v0
            self._limits = (v0, v1)
        def __call__(self, trial:optuna.trial):
            return trial.suggest_int(self._name, *self._limits)
    class Categorical(Abstract):
        def __init__(self, name:str, cats:Union[list[str],str], reds:Union[list[Any],Any]=None):
            super().__init__(name)
            if not isinstance(cats, list): cats = list(cats)
            if reds is not None and not isinstance(reds, list): reds = list(redirect)
            self._cats = cats
            self._reds = reds
        def __call__(self, trial:optuna.trial):
            val = trial.suggest_categorical(self._name, self._cats)
            if self._reds is not None:
                index = self._cats.index(val)
                val = self._reds[index]
            return val
    class Proportion(Abstract):
        def __init__(self, name:str, amount:int, nmap:list[str]=None):
            super().__init__(name)
            self._amount = amount
            self._nmap = nmap
        def __call__(self, trial:optuna.trial):
            coefficients = []
            for i in range(self._amount):
                addition = f'{i}' if self._nmap is None else self._nmap[i]
                coefficients.append(trial.suggest_float(f'{self._name}_{addition}', 0.0, 1.0))
            integral = sum(coefficients)
            coefficients = tuple([coefficient/integral for coefficient in coefficients])
            return coefficients
    
    def __class_getitem__(cls, key:Literal['float', 'int', 'cat']):
        if key == 'float': return Variation.Float
        if key == 'int':   return Variation.Int
        if key == 'cat':   return Variation.Categorical
        raise KeyError

# Настройки вариаций

In [6]:
# Параметры системы
var_wavelength   = Variation.Float('wavelength', 500.0E-9)
var_N            = Variation.Int('N', 5000) #5000
var_length       = Variation.Float('length', 2.5E-3*4)
var_pixels       = Variation.Int('pixels', 5000) #5000
var_distance     = Variation.Float('distance', 1.0E-3, 500.0E-3)
var_masks_amount = Variation.Int('masks', 1)
var_detectors_norm   = Variation.Categorical('detectors_norm', ['none', 'integral', 'minmax'])

# Параметры обучения
var_epochs           = Variation.Int('epochs', 5) #5
var_batch_per_device = Variation.Int('batch_per_device', 1, 8)
var_dataset_scale    = Variation.Float('dataset_scale', 0.1, 1.0)
var_loss_proportion  = Variation.Proportion('loss_proportion', 2, ['ce', 'mse'])
var_ce_norm          = Variation.Categorical('ce_norm',  ['minmax','max','softmax', 'none'], [Normalization.Minmax(), Normalization.Max(), Normalization.Softmax(), None])
var_mse_norm         = Variation.Categorical('mse_norm', ['minmax','max','softmax', 'none'], [Normalization.Minmax(), Normalization.Max(), Normalization.Softmax(), None])
var_optimizer        = Variation.Categorical('optimizer', ['Adam', 'SGD', 'RMSProp'], [torch.optim.Adam, torch.optim.SGD, torch.optim.RMSprop])
var_learning_rate    = Variation.Float('learning_rate', 1.0E-7, 2.0)

# Не менять
var_devices          = Variation.Int('devices', GPUCount)

# Определение функции оптимизации

In [7]:
def objective(trial:optuna.trial):
    wavelength = var_wavelength(trial)
    N = var_N(trial)
    length = var_length(trial)
    pixels = var_pixels(trial)
    distance = var_distance(trial)
    masks_amount = var_masks_amount(trial)
    detectors_norm = var_detectors_norm(trial)

    epochs = var_epochs(trial)
    devices = var_devices(trial)
    batch_per_device = var_batch_per_device(trial)
    batch_size = batch_per_device * devices
    dataset_scale = var_dataset_scale(trial)
    loss_proportion = var_loss_proportion(trial)
    ce_norm = var_ce_norm(trial)
    mse_norm = var_mse_norm(trial)
    optimizer = var_optimizer(trial)
    learning_rate = var_learning_rate(trial)

    print(f"Текущие параметры эксперимента: {trial.params}")
    
    propagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.4)
    phase_modulators = [PhaseModulator(N, length, pixels) for i in range(masks_amount)]
    amplitude_modulators = [AmplitudeModulator(N, length, pixels) for i in range(masks_amount)]
    elements = [propagation]
    for phase_modulator, amplitude_modulator in zip(phase_modulators, amplitude_modulators):
        elements += [phase_modulator, amplitude_modulator, propagation]
    spectral_filter = Window(centers=wavelength, sizes=300.0E-9)
    detectors_filter = Gaussian((length/50, length/50), (0,0))
    detectors = ClassificationDetectors(N, length, wavelength, 10, detectors_filter, spectral_filter)
    if detectors_norm == 'none':
        detectors.normalization.none()
    elif detectors_norm == 'integral':
        detectors.normalization.integral()
    elif detectors_norm == 'minmax':
        detectors.normalization.minmax()
    else: raise TypeError(f"Incorrect detectors normalization {detectors_norm}")
    model = CompositeModel(IntensityToAmplitude(), *elements, AmplitudeToIntensity(), detectors)
    
    dataset = Dataset('MNIST', batch_per_device, N, N, torch.float32, threads=GPUCount, preload=10)
    dataset.padding(surface_ratio=dataset_scale)
    loss_function = LossLinearCombination(Normalizable.CrossEntropy(ce_norm), Normalizable.MeanSquareError(mse_norm))
    loss_function.proportions(*loss_proportion)

    torch.cuda.empty_cache()
    SelectedGPUs.exclude(*ExcludedGPUs)
    mh, lh, cmh = cluster.epochs(epochs, 10, model, dataset, loss_function, optimizer, lr=learning_rate)

    accuracies = [100*numpy.sum(numpy.diagonal(confusion, 0)) / numpy.sum(confusion) for confusion in cmh]
    best_index = max(enumerate(accuracies), key=lambda x: x[1])[0]
    best_accuracy = accuracies[best_index]

    # model = pickle.dumps(mh[best_index-1]).hex()
    accuracy = json.dumps(best_accuracy)
    accuracies_history = json.dumps(accuracies)
    loss_history = json.dumps(numpy.concatenate(lh).tolist())
    confusion_matrixes = json.dumps(numpy.stack(cmh).tolist())

    # trial.set_user_attr('model', model)
    trial.set_user_attr('accuracy', accuracy)
    trial.set_user_attr('accuracies_history', accuracies_history)
    trial.set_user_attr('loss_history', loss_history)
    trial.set_user_attr('confusion_matrixes', confusion_matrixes)

    return best_accuracy
    
study = optuna.create_study(study_name="Default", storage="sqlite:///DNL.db", direction='maximize', load_if_exists=True)

[I 2024-10-04 09:31:43,049] Using an existing study with name 'Default' instead of creating a new one.


In [8]:
study.trials_dataframe().nlargest(20, 'value')

,number,value,datetime_start,datetime_complete,duration,params_N,params_batch_per_device,params_ce_norm,params_dataset_scale,params_detectors_norm,params_devices,params_distance,params_epochs,params_learning_rate,params_length,params_loss_proportion_ce,params_loss_proportion_mse,params_masks,params_mse_norm,params_optimizer,params_pixels,params_wavelength,user_attrs_accuracies_history,user_attrs_accuracy,user_attrs_confusion_matrixes,user_attrs_loss_history,state
119,119,97.070000,2024-10-01 13:21:04.258425,2024-10-01 15:32:22.810535,0 days 02:11:18.552110,5000,4,none,0.114897,none,8,0.497481,5,0.060592,0.01,0.211755,0.808082,1,minmax,RMSProp,5000,5.000000e-07,"[9.035, 95.20833333333333, 95.12166666666667, ...",97.07,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 740.375, 0.0, 0.0,...","[690185600.0, 5361039360.0, 1150803712.0, 2123...",COMPLETE
121,121,97.070000,2024-10-01 15:48:20.653704,2024-10-01 18:22:13.522311,0 days 02:33:52.868607,5000,4,none,0.120236,none,8,0.498786,5,0.192666,0.01,0.237850,0.812217,1,minmax,RMSProp,5000,5.000000e-07,"[9.035, 95.20833333333333, 95.12166666666667, ...",97.07,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 740.375, 0.0, 0.0,...","[690185600.0, 5361039360.0, 1150803712.0, 2123...",COMPLETE
122,122,96.280000,2024-10-01 18:22:14.606758,2024-10-01 19:50:41.291988,0 days 01:28:26.685230,5000,4,none,0.108823,none,8,0.499561,5,0.238423,0.01,0.234789,0.741893,1,minmax,RMSProp,5000,5.000000e-07,"[9.035, 92.43833333333333, 95.15666666666667, ...",96.28,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 740.375, 0.0, 0.0,...","[786754880.0, 3038854656.0, 2382456320.0, 1228...",COMPLETE
140,140,95.851667,2024-10-03 00:30:52.114040,2024-10-03 02:43:48.187041,0 days 02:12:56.073001,5000,4,none,0.163172,none,8,0.454843,5,0.216964,0.01,0.262099,0.806760,1,minmax,RMSProp,5000,5.000000e-07,"[9.035, 91.575, 94.47166666666666, 95.15833333...",95.85166666666667,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 740.375, 0.0, 0.0,...","[830090112.0, 3934009600.0, 4372558336.0, 1123...",COMPLETE
81,81,95.766667,2024-09-27 11:45:44.403863,2024-09-27 13:59:27.018576,0 days 02:13:42.614713,5000,2,none,0.172190,none,8,0.494349,5,0.248308,0.01,0.297126,0.306054,1,max,RMSProp,5000,5.000000e-07,"[9.035, 93.36, 94.81666666666666, 95.458333333...",95.76666666666667,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 740.375, 0.0, 0.0,...","[1770386176.0, 5445617664.0, 19652028416.0, 73...",COMPLETE
142,142,95.503333,2024-10-03 04:12:42.823857,2024-10-03 05:41:00.687034,0 days 01:28:17.863177,5000,4,none,0.154492,none,8,0.430652,5,0.305094,0.01,0.262568,0.812597,1,minmax,RMSProp,5000,5.000000e-07,"[9.035, 90.84333333333333, 93.82666666666667, ...",95.50333333333333,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 740.375, 0.0, 0.0,...","[834379584.0, 3537349120.0, 12481090560.0, 940...",COMPLETE
141,141,95.353333,2024-10-03 02:43:48.378481,2024-10-03 04:12:42.681005,0 days 01:28:54.302524,5000,4,none,0.155753,none,8,0.454246,5,0.232666,0.01,0.261913,0.810413,1,minmax,RMSProp,5000,5.000000e-07,"[9.035, 91.43333333333334, 94.01666666666667, ...",95.35333333333334,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 740.375, 0.0, 0.0,...","[827009664.0, 3212195328.0, 3325122816.0, 1634...",COMPLETE
133,133,95.238333,2024-10-02 12:42:44.174255,2024-10-02 14:11:02.037824,0 days 01:28:17.863569,5000,4,none,0.100806,none,8,0.444117,5,0.587960,0.01,0.267697,0.723360,1,minmax,RMSProp,5000,5.000000e-07,"[9.035, 91.06166666666667, 94.18, 94.778333333...",95.23833333333333,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 740.375, 0.0, 0.0,...","[890839488.0, 3753525504.0, 1895736832.0, 1626...",COMPLETE
124,124,95.210000,2024-10-01 21:20:05.077095,2024-10-01 22:48:26.992450,0 days 01:28:21.915355,5000,4,none,0.112174,none,8,0.498453,5,0.565737,0.01,0.241243,0.750750,1,minmax,RMSProp,5000,5.000000e-07,"[9.035, 91.02833333333334, 94.105, 94.785, 95....",95.21,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 740.375, 0.0, 0.0,...","[793387072.0, 3532347392.0, 1682785664.0, 1449...",COMPLETE
125,125,95.210000,2024-10-01 22:48:27.104323,2024-10-02 01:20:23.404618,0 days 02:31:56.300295,5000,4,none,0.105276,none,8,0.4666

In [9]:
acc_hists = study.trials_dataframe().nlargest(20, 'value')['user_attrs_accuracies_history']
for i, acc_hist in enumerate(acc_hists):
    acc_hist = json.loads(acc_hist)
    print(f"model {i}: " + (', '.join([f"{round(acc,1)}" for acc in acc_hist]) if isinstance(acc_hist, list) else f"{acc_hist}"))

model 0: 9.0, 95.2, 95.1, 95.7, 97.1, 97.1, 6.2
model 1: 9.0, 95.2, 95.1, 95.7, 97.1, 97.1, 6.2
model 2: 9.0, 92.4, 95.2, 95.8, 96.3, 13.3
model 3: 9.0, 91.6, 94.5, 95.2, 95.9, 95.9, 7.4
model 4: 9.0, 93.4, 94.8, 95.5, 95.8, 95.8, 32.9
model 5: 9.0, 90.8, 93.8, 95.0, 95.5, 12.6
model 6: 9.0, 91.4, 94.0, 94.6, 95.4, 14.0
model 7: 9.0, 91.1, 94.2, 94.8, 95.2, 7.7
model 8: 9.0, 91.0, 94.1, 94.8, 95.2, 9.8
model 9: 9.0, 91.0, 94.1, 94.8, 95.2, 9.8
model 10: 9.0, 91.0, 94.1, 94.8, 95.2, 9.8
model 11: 9.0, 91.7, 93.1, 94.4, 95.0, 20.0
model 12: 9.0, 91.6, 94.1, 94.7, 95.0, 12.0
model 13: 9.0, 87.7, 92.4, 94.8, 94.9, 15.3
model 14: 9.0, 86.0, 91.6, 92.2, 94.9, 4.1
model 15: 9.0, 90.0, 90.8, 93.9, 94.7, 9.6
model 16: 10.1, 85.0, 93.2, 94.7, 94.0, 1.2
model 17: 10.1, 85.0, 93.2, 94.7, 94.0, 1.2
model 18: 10.1, 85.0, 93.2, 94.7, 94.0, 1.2
model 19: 10.1, 85.0, 93.2, 94.7, 94.0, 1.2


# Оптимизация

In [ ]:
study.optimize(objective, n_trials=10)

Текущие параметры эксперимента: {'wavelength': 5e-07, 'N': 5000, 'length': 0.01, 'pixels': 5000, 'distance': 0.4783256792474156, 'masks': 1, 'detectors_norm': 'none', 'epochs': 5, 'devices': 8, 'batch_per_device': 4, 'dataset_scale': 0.11902440745527175, 'loss_proportion_ce': 0.2930206698117291, 'loss_proportion_mse': 0.7280264913654696, 'ce_norm': 'none', 'mse_norm': 'minmax', 'optimizer': 'RMSProp', 'learning_rate': 0.1050524557618314}
Training main thread PID is: 1715005
100%|███████████████████████████████████████| 1875/1875 [07:27<00:00,  4.19it/s]
Accuracy in the beginning is 9.035
RL:286.956·10³, RPI1K:-33.98·10⁶: 100%|███████████ | 1875/1875 | 08:20m - 00:00m
100%|███████████████████████████████████████| 1875/1875 [06:47<00:00,  4.60it/s]
Accuracy after epoch 1 is 93.83666666666667
RL:76.966·10³, RPI1K:-127.01·10³: 100%|███████████ | 1875/1875 | 08:05m - 00:00m
100%|███████████████████████████████████████| 1875/1875 [07:13<00:00,  4.33it/s]
Accuracy after epoch 2 is 96.015
RL:3

[I 2024-10-04 11:44:33,211] Trial 151 finished with value: 97.58333333333333 and parameters: {'wavelength': 5e-07, 'N': 5000, 'length': 0.01, 'pixels': 5000, 'distance': 0.4783256792474156, 'masks': 1, 'detectors_norm': 'none', 'epochs': 5, 'devices': 8, 'batch_per_device': 4, 'dataset_scale': 0.11902440745527175, 'loss_proportion_ce': 0.2930206698117291, 'loss_proportion_mse': 0.7280264913654696, 'ce_norm': 'none', 'mse_norm': 'minmax', 'optimizer': 'RMSProp', 'learning_rate': 0.1050524557618314}. Best is trial 151 with value: 97.58333333333333.


Текущие параметры эксперимента: {'wavelength': 5e-07, 'N': 5000, 'length': 0.01, 'pixels': 5000, 'distance': 0.4784265112755758, 'masks': 1, 'detectors_norm': 'none', 'epochs': 5, 'devices': 8, 'batch_per_device': 4, 'dataset_scale': 0.11769315595801616, 'loss_proportion_ce': 0.9922871384840444, 'loss_proportion_mse': 0.7161753092467288, 'ce_norm': 'none', 'mse_norm': 'minmax', 'optimizer': 'RMSProp', 'learning_rate': 0.0953193341096219}
Training main thread PID is: 1846931
 41%|████████████████▎                       | 765/1875 [03:02<03:58,  4.66it/s]

In [ ]:
study.optimize(objective, n_trials=10)

In [ ]:
study.optimize(objective, n_trials=10)